In [1]:
from math import *
import numpy as np
import numpy.random as rd
import PredDens 

Demo (Table 1 in Yano, Kaneko, Komaki)

We use the following evaluation metrics:
- $\ell_1$-distance
- Weighted $\ell_1$-distance
- Predictive coverage

In [2]:
#Required functions

def diff_l1(v1,v2):
    return sum([abs(v1[i]-v2[i]) for i in range(len(v1))])

def wdiff_l1(v1, v2, r):
    return sum([abs(v1[i]-v2[i])*r[i]/np.average(r) for i in range(len(v1))])

def JointCI_count(future, alpha, Y, mean, r):
    # future : samples drawn from the model. shape = (num_future, dim)
    # Y : samples drawn from a predictive density. shape = (N, dim)
    # mean : mean of a predictive density
    # reduce a multi-dimension into one dimension value with weighted L1 distance
    # count the number of  future observations that are in the joint CI in the sence of above L1 ball 
    dist = [PredDens.weighted_L1(Y[i,:], mean, r) for i in range(0, len(Y))]
    a = [PredDens.weighted_L1(future[i,:], mean, r) for i in range(0, len(future))]
    return sum([(0 <= a[i] and a[i] <= np.percentile(dist, alpha * 100)) for i in range(0, len(future))])

#optimal scaling in Proposition 2.5
scaling_denom = lambda r, kappa: (r**(-kappa))*(1-(r/(r+1))**kappa)/kappa
scaling_nom = lambda r: ((r/(r+1))**r)*(1/(r+1))
scaling = lambda r, kappa: np.average(scaling_nom(r))/np.average(scaling_denom(r, kappa))

In [3]:
# random sample generation

dim = 200  # dimension of simulated data
s = 5    # sparsity level (the number of nonzero data)
num_current = 1   # the number of current observation
r = num_current * np.ones(dim)
num_future = 500
sample = np.zeros((num_current, dim)) 
future = np.zeros((num_future, dim))
nonzero_id = [floor(rd.rand() * dim) for i in range(0, s)]   # indice of nonzero values in one sample
nonzero_val = rd.gamma(shape = 10, scale = 1, size = len(nonzero_id))
para = np.zeros(dim)
for i in range(0, s):
    para[nonzero_id[i]] = nonzero_val[i]
    
NN =500 # the number of changing current observations

In [4]:
# Initialization
L1Dist_Proposed, logl_Proposed, CICount_Proposed_09 = np.zeros(NN),np.zeros(NN),np.zeros(NN)

In [5]:
N = 200 # number of samples generated by each method
alpha = 0.9

for j in range(0, NN):
    for i in range(0, s):
        sample[:, nonzero_id[i]] = rd.poisson(lam = nonzero_val[i], size = num_current)
        future[:, nonzero_id[i]] = rd.poisson(lam = nonzero_val[i], size = num_future)
        
    x = np.sum(sample, axis = 0)
    s_hat = len(np.nonzero(x)[0])
    kappa = 0.1
    ins_Proposed = PredDens.ProposedPredictiveDensity(eta = scaling(r, kappa) * s_hat / dim, dim = dim, r =r, x = x, kappa = kappa)   
    sample_Proposed = ins_Proposed.sample_gen(num = N)
    mean_Proposed = np.average(sample_Proposed, axis = 0)
    L1Dist_Proposed[j] = diff_l1(mean_Proposed, future[0,:])
    logl_Proposed[j] = ins_Proposed.log_likelihood(future[0, :])
    CICount_Proposed_09[j] = JointCI_count(future, alpha, sample_Proposed, mean_Proposed, r) 

In [6]:
print("Results:")
print("  l1 distance                                              = ", np.round(np.average(L1Dist_Proposed),1), "(", np.round(np.std(L1Dist_Proposed),1), ")")
print("  log predictive likelihood                        = ", np.round(np.average(logl_Proposed),1), "(", np.round(np.std(logl_Proposed),1), ")")
print("  90 % predictive coverage probability = ", np.round(np.average(CICount_Proposed_09) / num_future,3), "(", np.round(np.std(CICount_Proposed_09) / num_future,2), ")\n")

Results:
  l1 distance                                              =  17.5 ( 5.4 )
  log predictive likelihood                        =  -15.1 ( 1.7 )
  90 % predictive coverage probability =  0.929 ( 0.1 )

